In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent.absolute()))
import config
import yaml

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_token_details_for_address, get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
from contracts.guild_contract import GuildContract
import utilities

context = Context()

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = utilities.get_default_data()

2024-07-31 01:42:03,355 - WARNING - Structure definition for stakings_boosted not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json


loaded 1 accounts from 1 PEM files.


2024-07-31 01:42:03,756 - DEBUG - Account.sync_nonce() done: 3614
2024-07-31 01:42:03,962 - DEBUG - Account.sync_nonce() done: 3614
2024-07-31 01:42:03,963 - INFO - Loaded 10 tokens.
2024-07-31 01:42:04,163 - DEBUG - Account.sync_nonce() done: 3614
2024-07-31 01:42:04,163 - INFO - Loaded 0 stakings.
2024-07-31 01:42:04,164 - INFO - Loaded 7 stakings_v2.
2024-07-31 01:42:04,165 - INFO - Loaded 0 stakings_boosted.
2024-07-31 01:42:04,351 - DEBUG - Account.sync_nonce() done: 3614
2024-07-31 01:42:04,351 - INFO - Deployed contracts below:
2024-07-31 01:42:04,352 - INFO - stakings:
2024-07-31 01:42:04,352 - INFO - stakings_v2:
2024-07-31 01:42:04,353 - INFO - Deployed staking contract: erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7
2024-07-31 01:42:04,354 - INFO -  ├ Staking token: RIDE-7d18e9
2024-07-31 01:42:04,354 - INFO -  ├ Stake token: SRIDE-4ab1d4
2024-07-31 01:42:04,355 - INFO - Deployed staking contract: erd1qqqqqqqqqqqqqpgqr7kdhagkqgxvjrsk7s5333l9wwnenr9g2jps8puq33

In [4]:
user2 = Account(pem_file="/home/multiversx/Documents/guilds_py/wallets/wallet2.pem")
user2.address = Address("erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz")

user = Account(pem_file="/home/multiversx/Documents/guilds_py/wallets/wallet1.pem")
user.address = Address("erd1a2x0azarwsag9svwdn23pjramat7zcjceqd44yh4g7jx4pugkg2s3jwlel")
user.sync_nonce(context.network_provider.proxy)

2024-07-31 01:42:38,619 - DEBUG - Account.sync_nonce() done: 62


In [ ]:
from utils.contract_retrievers import retrieve_guild_by_address


with open("config_file.yaml", "r") as file:
    config = yaml.safe_load(file)

guild_contract: GuildContract = retrieve_guild_by_address(config["deployedGuildAddress"])
guild_address = Address(config["deployedGuildAddress"])

ADVANCE EPOCHS

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, 1, 20)

In [ ]:
import time
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration


token = "TESTUTK-cdbafa"
holder = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
holder.address = Address("erd1a2x0azarwsag9svwdn23pjramat7zcjceqd44yh4g7jx4pugkg2s3jwlel")
user.sync_nonce(context.network_provider.proxy)
holder.sync_nonce(context.network_provider.proxy)
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))


hashes = []
configs = DefaultTransactionBuildersConfiguration(chain_id="1")


transfers = [
    TokenPayment.meta_esdt_from_amount(token, 0, 100000, 18)
             ]

builder = MultiESDTNFTTransferBuilder(
        configs,
        destination=user.address,
        sender=holder.address,
        gas_limit=30000000,
        payments=transfers
    )

tx = builder.build()
tx.nonce = holder.nonce
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)
print(hash)
hashes.append(hash)
time.sleep(2)

ISSUE TOKENS

In [ ]:
import time
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
nr_tokens = 1
token_ticker = "TESTUTK"
holder = Account(pem_file="/home/multiversx/Documents/guilds_py/defi-wallet.pem", pem_index=0)
holder.address = Address("erd1a2x0azarwsag9svwdn23pjramat7zcjceqd44yh4g7jx4pugkg2s3jwlel")
holder.sync_nonce(context.network_provider.proxy)

# issue a number of nr_tokens tokens
hashes = []
# for i in range(1, nr_tokens+1):
hash = esdt_contract.issue_fungible_token(holder, context.network_provider.proxy, 
                                     [f"{token_ticker}", f"{token_ticker}", 1000000000000000000000000000, 18, 
                                      "canMint", "true", "canBurn", "true", "canPause", "true", "canFreeze", "true", "canWipe", "true", "canChangeOwner", "true", "canUpgrade", "true", "canAddSpecialRoles", "true"])
print(f"Issued {token_ticker} in: {hash}")
hashes.append(hash)
time.sleep(1)

# check issue success and retrieve tokens
issued_tokens = []
for hash in hashes:
    if context.network_provider.check_complex_tx_status(hash, "issue token"):
        tx_data = context.network_provider.proxy.get_transaction(hash)
        issue_event = tx_data.logs.find_first_or_none_event("issue")
        issued_token = str(issue_event.topics[0])
        print(issue_event.topics[0].raw)
        print("Issued token:", issued_token)
        issued_tokens.append(issued_token)
    else:
        print(f"Transaction {hash} failed!")

print(f'Issued ESDT Tokens: {issued_tokens}')

GUILD QUERY

In [ ]:
from multiversx_sdk import SmartContractTransactionsFactory
from utilities import get_default_data
from utils.contract_data_fetchers import GuildContractDataFetcher

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()

# context.network_provider.proxy.get_data('getFarmTokenId')
guild_contract_address = Address(configuration["guildScAddressModel"])
GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getFarmingTokenId")


PAUSE

In [ ]:

from contracts.farm_contract import FarmContract
# from contracts.guild_contract import GuildContract
user.sync_nonce(context.network_provider.proxy)

guild_contract: FarmContract = retrieve_farm_by_address(config["guildScAddressModel"])

tx_hash = guild_contract.pause(user, context.network_provider.proxy)

RESUME

In [ ]:
import yaml
from contracts.guild_contract import GuildContract


with open("config_file.yaml", "r") as file:
    config_file = yaml.safe_load(file)


user.sync_nonce(context.network_provider.proxy)
context.deployer_account.sync_nonce(context.network_provider.proxy)

guild_contract: GuildContract = retrieve_farm_by_address(config_file["deployedGuildAddress"])

tx_hash = guild_contract.resume(user, context.network_provider.proxy)

START PRODUCE REWARDS

In [ ]:
import yaml
from contracts.guild_contract import GuildContract


with open("config_file.yaml", "r") as file:
    config_file = yaml.safe_load(file)

context.deployer_account.sync_nonce(context.network_provider.proxy)

guild_contract: GuildContract = retrieve_farm_by_address(config_file["deployedGuildAddress"])

tx_hash = guild_contract.start_produce_rewards(user, context.network_provider.proxy)

REGISTER FARM TOKEN

In [ ]:
import yaml
import time
from utilities import tx_success, get_default_data
from utils.contract_data_fetchers import FarmContractDataFetcher
from multiversx_sdk import TransactionComputer

guild_address = Address(config_file["deployedGuildAddress"])
token = "TESTUTK-cdbafa"

(
    config,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()


register_farm_token_tx = sc_factory.create_transaction_for_execute(
    sender=test_address,
    contract=guild_address,
    function="registerFarmToken",
    gas_limit=70000000,
    arguments=[],
    # "UTKFARM", "UTKFARM", 18,""
    native_transfer_amount=50000000000000000
 
)
register_farm_token_tx.nonce = nonce_holder.get_nonce_then_increment()
register_farm_token_tx.signature = signer.sign(
    TransactionComputer().compute_bytes_for_signing(register_farm_token_tx)
)

result = provider.send_transaction(register_farm_token_tx)
print("Register Farm Token transaction hash:", result)
time.sleep(3)
tx_status = tx_success(result, provider)
if not tx_status:
    print("Register Farm Token transaction failed")
else:
    print("Farm token registered successfully")


Set admin

In [ ]:
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner


newAdmin = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
newAdmin = Address("erd1qrat8pxccn8mj7n47v38uw7q4nkej6zq6mny3ljkj29uzdf6s42srfja7q")
config = DefaultTransactionBuildersConfiguration(chain_id="1")
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))
user.sync_nonce(context.network_provider.proxy)

builder = ContractCallBuilder(
    config,
    contract=Address(guild_contract.address),
    function_name="addAdmin",
    caller=user.address,
    nonce=user.nonce,
    call_arguments=[newAdmin],
    gas_limit=30000000
)

tx = builder.build()
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)


In [ ]:
#GUILD FACTORY QUERIES
from multiversx_sdk import SmartContractTransactionsFactory
from utilities import get_default_data
from utils.contract_data_fetchers import GuildFactoryDataFetcher

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()

# context.network_provider.proxy.get_data('getFarmTokenId')
guild_contract_address = Address(configuration["guildFactoryScAddress"])
GuildFactoryDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getConfigAddress")


TRANSFER TOKENS

In [ ]:
import time
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration


token = "UTKFARMN-535c6d"
holder = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
holder.address = Address("erd1a2x0azarwsag9svwdn23pjramat7zcjceqd44yh4g7jx4pugkg2s3jwlel")
user2.address = Address("erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz")
user2.sync_nonce(context.network_provider.proxy)
holder.sync_nonce(context.network_provider.proxy)
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))


hashes = []
configs = DefaultTransactionBuildersConfiguration(chain_id="1")


transfers = [
    TokenPayment.meta_esdt_from_amount(token, 1, 500000, 18)
             ]

builder = MultiESDTNFTTransferBuilder(
        configs,
        destination=user2.address,
        sender=holder.address,
        gas_limit=30000000,
        payments=transfers
    )

tx = builder.build()
tx.nonce = holder.nonce
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)
print(hash)
hashes.append(hash)
time.sleep(2)

STAKE

In [ ]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent
from utils.utils_tx import ESDTToken

guild_address = Address(config["deployedGuildAddress"])

# user.address = Address("erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz")
user2.sync_nonce(context.network_provider.proxy)

farming_tk_balance = 0
farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user2.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

event = EnterFarmEvent(guild_contract.farmingToken, 0, 10000000000000000000000,
                       guild_contract.farmToken, farm_tk_nonce, farm_tk_balance)

tx = guild_contract.stakeFarm(
    context.network_provider,
    user2, 
    event,
    True
)

print("Transaction hash:", tx)

UNSTAKE FROM GUILD

In [10]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken

guild_address = Address(configuration["deployedGuildAddress"])

user.address = Address("erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz")
user.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address, context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
print(tokens_in_account)
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        event = ExitFarmEvent(guild_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')
        tx_hash = guild_contract.exitFarm(context.network_provider.proxy, user, event)
        # break


# 

TypeError: 'module' object is not subscriptable

UNBOND FARM

In [ ]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher


unbond_token = hex_to_string(GuildContractDataFetcher(guild_address,context.network_provider.proxy.url).get_data("getUnbondTokenId"))

# user.address = Address("erd1kvy5mas5z5ul3tt0ejvtrqcnrqqq6nwmwlxu7xpkqysnp3xwc06sgja0vu")
user.sync_nonce(context.network_provider.proxy)


unbond_tk_balance, unbond_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(unbond_token, user.address, context.network_provider.proxy)
print(tokens_in_account)

print(f'Found {len(tokens_in_account)} positions of {unbond_token} in account')
for token in tokens_in_account:
    if int(token['balance']) > unbond_tk_balance:
        unbond_tk_balance = int(token['balance'])
        unbond_tk_nonce = token['nonce']
        event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')
        tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)
        # break

print(unbond_tk_balance)

# event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')

# tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)

CLAIM REWARDS

In [8]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user.address = Address("erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz")

user2.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user2.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

        tx_hash = guild_contract.claimRewards(context.network_provider, user2, event)
        # break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")



2024-07-31 08:49:20,765 - DEBUG - Account.sync_nonce() done: 4
2024-07-31 08:49:21,978 - INFO - claimRewards
2024-07-31 08:49:21,979 - DEBUG - Account: erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz
2024-07-31 08:49:21,979 - DEBUG - claimRewards
2024-07-31 08:49:22,170 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.


Found 2 positions of UTKFARMN-535c6d in account


2024-07-31 08:49:22,366 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/a8fe31677af0a6937f9fd99e1565f0e5740a1c697cef018cefee954893d984e7
2024-07-31 08:49:22,367 - INFO - claimRewards
2024-07-31 08:49:22,367 - DEBUG - Account: erd1upgm0ghcvsmnup2a22pjrla7turpp8mvzn5jqh0ahrsrv0tegypqmphthz
2024-07-31 08:49:22,368 - DEBUG - claimRewards
2024-07-31 08:49:22,553 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2024-07-31 08:49:22,762 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/8f93323b5e78b50fbb9068e8acba3f72d6d3b3e1fd2762e5b9447d3b160e0706


COMPOUND REWARDS

In [ ]:
from events.farm_events import CompoundRewardsFarmEvent
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)


farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        # break
        event = CompoundRewardsFarmEvent(farm_tk_balance, farm_tk_nonce)

tx_hash = guild_contract.compoundRewards(context.network_provider, user, event)

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")



MERGE TOKENS

In [ ]:
from events.farm_events import CompoundRewardsFarmEvent
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)


farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = CompoundRewardsFarmEvent(1000000000, farm_tk_nonce)

tx_hash = guild_contract.compoundRewards(context.network_provider, user, event)

CALCULATE REWARDS FOR USER

In [7]:
from utils import contract_retrievers
from utils.utils_chain import Account, WrapperAddress as Address, base64_to_hex, decode_merged_attributes, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher
from contracts.guild_contract import GuildContract
from multiversx_sdk.abi.biguint_value import BigUIntValue
from multiversx_sdk.abi.bytes_value import BytesValue
from multiversx_sdk.abi.fields import Field
from multiversx_sdk.abi.serializer import Serializer
from multiversx_sdk.abi.small_int_values import U64Value, U32Value
from multiversx_sdk.abi.struct_value import StructValue
from multiversx_sdk import SmartContractTransactionsFactory, TransactionComputer
from utilities import get_default_data
from utils.contract_data_fetchers import GuildFactoryDataFetcher

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()
guild_contract: GuildContract = contract_retrievers.retrieve_guild_by_address(configuration["deployedGuildAddress"])

user.sync_nonce(context.network_provider.proxy)
guild_contract_address = Address(configuration["deployedGuildAddress"])



farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    # print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
attr = [0,0,0]


for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']


rewards_per_share = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getUserRewardPerShare")
compounded_reward = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getAccumulatedRewards")
current_farm_amount = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getFarmTokenSupply")



# attributes_hex = base64_to_hex(tokens[token]['attributes'])
# decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), FARM_TOKEN_ATTRIBUTES)


# tx_hash = guild_contract.calculateRewardsForUser(context.network_provider, user, farm_tk_balance, attr) #hex value

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")
print(rewards_per_share)
print(compounded_reward)
print(current_farm_amount)

serializer = Serializer(parts_separator="@")

staking_token_attributes = StructValue(
    fields=[
        Field(name="reward_per_share", value=BigUIntValue(100000000000000000000)),
        Field(name="compounded_reward", value=BigUIntValue(100000000000000000000)),
        Field(name="current_farm_amount", value=BigUIntValue(100000000000000000000)),
    ]
)

bytecode = Path(
    "/home/multiversx/Documents/guilds_py/wasm/guild-sc.wasm"
).read_bytes()

deploy_config_sc_data = serializer.serialize_to_parts([staking_token_attributes, BytesValue(bytecode)])


calc_rewards_for_user_tx = sc_factory.create_transaction_for_execute(
    sender=test_address,
    contract=guild_contract_address,
    function="calculateRewardsForGivenPosition",
    gas_limit=70000000,
    arguments=[test_address, 100000, deploy_config_sc_data]
)

calc_rewards_for_user_tx.nonce = nonce_holder.get_nonce_then_increment()
calc_rewards_for_user_tx.signature = signer.sign(
    TransactionComputer().compute_bytes_for_signing(calc_rewards_for_user_tx)
)

result = provider.send_transaction(calc_rewards_for_user_tx)


2024-07-31 01:46:29,875 - DEBUG - Account.sync_nonce() done: 62


Farm Tokens in account:
	{'attributes': 'AAAAAAAAAAAAAAAKaeEN52Z20IAAAA==', 'balance': '500000000000000000000000', 'creator': 'erd1qqqqqqqqqqqqqpgq0sqcgdlgt5mrp3l4qhw6mzpu3z45778qkg2sx8xm2q', 'nonce': 1, 'royalties': '0', 'tokenIdentifier': 'UTKFARMN-535c6d-01', 'uris': ['']}
Found 1 positions of UTKFARMN-535c6d in account
145928462709282
14114098173515980616
511000000000000000000000


ErrCannotSerializeArgument: Cannot serialize: [b'\x00\x00\x00\t\x05k\xc7^-c\x10\x00\x00\x00\x00\x00\t\x05k\xc7^-c\x10\x00\x00\x00\x00\x00\t\x05k\xc7^-c\x10\x00\x00', b'\x00asm\x01\x00\x00\x00\x01\xcc\x01\x1f`\x02\x7f\x7f\x00`\x00\x00`\x00\x01\x7f`\x01\x7f\x01\x7f`\x01\x7f\x00`\x02\x7f\x7f\x01\x7f`\x03\x7f\x7f\x7f\x00`\x04\x7f\x7f\x7f\x7f\x00`\x03\x7f\x7f\x7f\x01\x7f`\x04\x7f\x7f\x7f\x7f\x01\x7f`\x02\x7f~\x00`\x00\x01~`\x01\x7f\x01~`\x02\x7f\x7f\x01~`\x05\x7f\x7f\x7f\x7f\x7f\x00`\x02\x7f~\x01\x7f`\x04\x7f~\x7f\x7f\x00`\x01~\x01\x7f`\x05\x7f\x7f~\x7f\x7f\x01\x7f`\x01~\x00`\x03\x7f~\x7f\x00`\x04\x7f\x7f\x7f~\x00`\x05~\x7f\x7f\x7f\x7f\x01\x7f`\x06~\x7f\x7f\x7f\x7f\x7f\x01\x7f`\x0b\x7f\x7f~\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x00`\x02~\x7f\x00`\x03\x7f\x7f\x7f\x01~`\x03\x7f\x7f~\x00`\x06\x7f\x7f\x7f\x7f\x7f\x7f\x00`\x05\x7f\x7f\x7f\x7f~\x00`\x04\x7f~\x7f\x7f\x01\x7f\x02\xb1\n7\x03env\x0ebigIntSetInt64\x00\n\x03env\tbigIntAdd\x00\x06\x03env\x0bsignalError\x00\x00\x03env\nmBufferNew\x00\x02\x03en

UPGRADE

In [ ]:
from multiversx_sdk import CodeMetadata
from utils.utils_tx import upgrade_call


guild_address = Address.from_bech32(configuration["guildConfigScAddressModel"])

context.deployer_account.address = user.address
context.deployer_account.sync_nonce(context.network_provider.proxy)
# hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.DUMMY_PROXY, [])

metadata = CodeMetadata(upgradeable=True, payable_by_contract=True, readable=True)
gas_limit = 300000000

# if no_init:
#         arguments = []
# else:
arguments = []

bytecode = Path(
    "/home/multiversx/Documents/guilds_py/wasm/guild-sc-config.wasm"
    )

tx_hash = upgrade_call("guild", context.network_provider.proxy, gas_limit, context.deployer_account, guild_address,
                               bytecode, metadata, arguments)


END REWARDS

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.end_produce_rewards(user, context.network_provider.proxy)

In [ ]:
#GET GUILD CONFIG STATS

#GET GUILD FACTORY STATS

#GET GUILD SC STATS

guild_stats = guild_contract.get_all_stats(context.network_provider.proxy)

print(guild_stats)


# farm_token_nonce, farm_token_amount = get_token(proxy_contract.proxy_farm_token, int(looking_for_nonce, 16))

# farm_dec_attributes, lp_dec_attributes, lktk_dec_attributes = guild_contract.get_all_decoded_farm_token_attributes_from_api(context.network_provider.proxy, get_nonce(guild_contract.address))


CANCEL UNBOND FARM

In [ ]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher


unbond_token = hex_to_string(GuildContractDataFetcher(guild_address,context.network_provider.proxy.url).get_data("getUnbondTokenId"))

# user.address = Address("erd1kvy5mas5z5ul3tt0ejvtrqcnrqqq6nwmwlxu7xpkqysnp3xwc06sgja0vu")
user.sync_nonce(context.network_provider.proxy)


unbond_tk_balance, unbond_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(unbond_token, user.address, context.network_provider.proxy)
print(tokens_in_account)

print(f'Found {len(tokens_in_account)} positions of {unbond_token} in account')
for token in tokens_in_account:
    if int(token['balance']) > unbond_tk_balance:
        unbond_tk_balance = int(token['balance'])
        unbond_tk_nonce = token['nonce']
        event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')
        tx_hash = guild_contract.cancelUnbondFarm(context.network_provider.proxy, user, event)
        # break

print(unbond_tk_balance)

# event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')

# tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)

SET MIN/MAX STAKE

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.set(user, context.network_provider.proxy)

CHECK GUILD

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.is_guild_closing(user, context.network_provider.proxy)

TRANSFER TOKENS TO FACTORY

In [ ]:
from multiversx_sdk import ProxyNetworkProvider, Token, TokenTransfer

from utilities import to_decimal

# Signing wallet bech32 address
test_address = Address.from_bech32(config["signingWalletAddress"])
guild_factory_address = Address.from_bech32(config["guildFactoryScAddress"])

# Network provider
provider = ProxyNetworkProvider(config["defaultPROXY"])

# Get account for the signing wallet
test_account = provider.get_account(test_address)

# Get network configuration
network_config = provider.get_network_config()

# Load the signer from the pem file for the signing wallet
signer = UserSigner.from_pem_file(
    Path(config["signingWalletPemPath"]),
)

# Nonce holder
nonce_holder = AccountNonceHolder(test_account.nonce)

farming_token = Token(config["farmingToken"])
farming_token_transfer = TokenTransfer(farming_token, to_decimal(10000, 18))
transfer_rewards_tx = sc_factory.create_transaction_for_execute(
    sender=test_address,
    contract=guild_factory_address,
    function="depositRewardsAdmins",
    gas_limit=10000000,
    token_transfers=[farming_token_transfer],
)

transfer_rewards_tx.nonce = nonce_holder.get_nonce_then_increment()

transfer_rewards_tx.signature = signer.sign(
    TransactionComputer().compute_bytes_for_signing(transfer_rewards_tx)
)

result = provider.send_transaction(transfer_rewards_tx)

TOPUP REWARDS

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent
from utilities import to_decimal


# user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.topup_rewards(user, context.network_provider.proxy, to_decimal(10000, 18))